## Korean_food Multi Label Classification

In [1]:
import os
import zipfile
from zipfile import ZipFile
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [2]:
# 데이터 종류별 폴더
data_dir='c:/data/kfood'

In [3]:
# 압축 해제
# folderList=glob(data_dir + '/*')
# total=0
# for folder  in folderList:
#     print(folder)
#     zipfile.ZipFile(folder).extractall()
    # if os.path.isfile(folder):
    #     os.remove(folder)

############## 폴더 항목별 개수 확인
# for folder  in folderList:
#     print(folder)
#     fileList=glob(folder + '/*' )
#     # fileList=glob(folder + '/*.jpg' )
#     print(len(fileList))
#     total+=len(fileList)

# print(total)

### masterCategory, subCategory, image_ID 정보를 담은 csv 파일 생성

In [4]:
from glob import glob
root='c:/data/kfood/*'
folderList=glob(root)
y1,y2,y3=[],[],[]

for folder in folderList:
    subfolderList=glob(folder + '/*')

    for sub in subfolderList:
        fileList=glob(sub + '/*.jpg')

        for file in fileList:
            x=file.split('\\')
            y1.append(x[1])
            y2.append(x[2])
            y3.append(x[3])
           

In [5]:
len(y1),len(y2),len(y3)

(14881, 14881, 14881)

In [6]:
y1[20000:20010],y2[20000:20010]

([], [])

In [7]:
import pandas as pd

food_df = pd.DataFrame(y1, columns=['masterCategory'])
food_df['subCategory'] = y2
food_df['image_ID'] = y3
food_df.to_csv('k-food.csv', index=False)

## Data Load

In [8]:
food_df.head()

,masterCategory,subCategory,image_ID
0,구이,갈비구이,Img_000_0000.jpg
1,구이,갈비구이,Img_000_0001.jpg
2,구이,갈비구이,Img_000_0002.jpg
3,구이,갈비구이,Img_000_0003.jpg
4,구이,갈비구이,Img_000_0004.jpg


In [9]:
len(food_df)

14881

In [10]:
for cnt,name in enumerate(food_df.image_ID):
        src=str(data_dir+'/'+y1[cnt]+'/'+y2[cnt]+'/'+name)
        print(src)

c:/data/kfood/구이/갈비구이/Img_000_0000.jpg
c:/data/kfood/구이/갈비구이/Img_000_0001.jpg
c:/data/kfood/구이/갈비구이/Img_000_0002.jpg
c:/data/kfood/구이/갈비구이/Img_000_0003.jpg
c:/data/kfood/구이/갈비구이/Img_000_0004.jpg
c:/data/kfood/구이/갈비구이/Img_000_0005.jpg
c:/data/kfood/구이/갈비구이/Img_000_0006.jpg
c:/data/kfood/구이/갈비구이/Img_000_0007.jpg
c:/data/kfood/구이/갈비구이/Img_000_0008.jpg
c:/data/kfood/구이/갈비구이/Img_000_0009.jpg
c:/data/kfood/구이/갈비구이/Img_000_0010.jpg
c:/data/kfood/구이/갈비구이/Img_000_0011.jpg
c:/data/kfood/구이/갈비구이/Img_000_0012.jpg
c:/data/kfood/구이/갈비구이/Img_000_0013.jpg
c:/data/kfood/구이/갈비구이/Img_000_0014.jpg
c:/data/kfood/구이/갈비구이/Img_000_0015.jpg
c:/data/kfood/구이/갈비구이/Img_000_0016.jpg
c:/data/kfood/구이/갈비구이/Img_000_0017.jpg
c:/data/kfood/구이/갈비구이/Img_000_0018.jpg
c:/data/kfood/구이/갈비구이/Img_000_0019.jpg
c:/data/kfood/구이/갈비구이/Img_000_0020.jpg
c:/data/kfood/구이/갈비구이/Img_000_0021.jpg
c:/data/kfood/구이/갈비구이/Img_000_0022.jpg
c:/data/kfood/구이/갈비구이/Img_000_0023.jpg
c:/data/kfood/구이/갈비구이/Img_000_0024.jpg
c:/data/kfood/구이/갈비구이/Img

In [11]:
data = []

from tensorflow.keras.preprocessing.image import img_to_array
import cv2

IX = 140
IY = 140

# for cnt,name in enumerate(df.image_ID):
#     path = data_dir+str(df.masterCategory[cnt])+'/'+str(df.subCategory[cnt])+'/'+df.image_ID
    
#     img_array = np.fromfile(path, np.uint8)
#     img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
#     image = cv2.resize(img, (img_height,img_width) )
#     image = img_to_array(image)
#     data.append(image)

# for cnt,name in enumerate(food_df.image_ID):
#         src=str(data_dir+'/'+y1[cnt]+'/'+y2[cnt]+'/'+name)
#         img_array = np.fromfile(src, np.uint8)
#         image = cv2.imdecode(img_array,cv2.IMREAD_COLOR)
#         image = cv2.resize(image, (IX,IY))
#         #plt.imshow('image',image)
#         image = img_to_array(image)
#         #print('toarray ',image)

#         data.append(image)        

invalid_ids = []

for cnt,name in enumerate(food_df.image_ID):

    try:
        src=str(data_dir+'/'+y1[cnt]+'/'+y2[cnt]+'/'+name)
        img_array = np.fromfile(src, np.uint8)
        image = cv2.imdecode(img_array,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IX,IY))
        image = img_to_array(image)
        data.append(image)
        
    except: 
        # Images for certain ids are missing, so they are not added to the dataset  
        invalid_ids.append(name)

In [12]:
print(len(invalid_ids))
print(len(data))

45
14836


In [13]:
labels = []
used_columns=['masterCategory','subCategory']

for index, row in food_df.iterrows():
    if row['image_ID'] not in invalid_ids:

        tags = []

        for col in used_columns:
            tags.append(row[col])

        labels.append(tags)

# for index, row in food_df.iterrows():
#     if row['image_ID'] in invalid_ids:
#         continue

#     tags = []

#     for col in used_columns:
#         tags.append(row[col])

#     labels.append(tags)

In [14]:
len(data), len(labels)

(14836, 14836)

In [15]:
# converting data into numpy arrays

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

print(labels)

MemoryError: Unable to allocate 6.50 GiB for an array with shape (14836, 140, 140, 3) and data type float64

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# creating a binary vector for the input labels 
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(mlb.classes_)
print(labels[0])

['갈비구이' '곱창구이' '구이' '기타' '닭갈비' '라면' '막국수' '면' '물냉면' '비빔냉면' '삼겹살' '양념치킨'
 '조개구이' '짜장면' '짬뽕' '쫄면' '피자' '후라이드치킨']
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Modeling

##### Model_1

In [ ]:
data.shape, labels.shape

In [ ]:
from sklearn.model_selection import train_test_split

(X_train, X_eval, y_train, y_eval) = train_test_split(data,labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

inputShape = (IY, IX, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten()) 

model.add(Dense(128))
model.add(Activation('sigmoid'))


out = len(mlb.classes_)

model.add(Dense(out))
model.add(Activation('sigmoid')) 
                    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['mse'])

In [ ]:
batch = 32
E = 30

############### checkpoint ############### 
from tensorflow.keras.callbacks import ModelCheckpoint
modelpath="c:/data/model/k-food_bestmodel.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, verbose=1, monitor='val_loss', save_best_only=True) 

def scheduler(epoch, learning_rate):
    if epoch < 10:
        return learning_rate
    else:
        return learning_rate * tf.math.exp(-0.1)
lr_scheduler_cb = tf.keras.callbacks.LearningRateScheduler(scheduler) 

#training the model 
history=model.fit(X_train,
                  y_train,
                  epochs=E,
                  verbose=1,
                  callbacks=[checkpointer,lr_scheduler_cb])

In [ ]:
import pandas as pd
hist_df=pd.DataFrame(history.history)
hist_df.head()

In [ ]:
# preds = model.predict(X_eval)
# print(preds)

In [ ]:
preds = model.predict(X_eval)

# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized = []

for pred in preds:
    vals = []
    for val in pred:
        if val > 0.25:
            vals.append(1)
        else:
            vals.append(0)
    pred_binarized.append(vals) 

pred_binarized = np.array(pred_binarized)   

# we convert the output vectors to the predicted labels
true_test_labels = mlb.inverse_transform(y_eval)
pred_test_labels = mlb.inverse_transform(pred_binarized)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(y_eval)):

    true_labels = list(true_test_labels[i])

    pred_labels = list(pred_test_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]

    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    



print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

In [ ]:
for i in range(10):
    print('True labels: ',true_test_labels[i],' Predicted labels: ',pred_test_labels[i])